In [ ]:
# Initialize Otter
!pip install otter-grader
import otter
grader = otter.Notebook("scraping_assignment.ipynb")

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 4.7 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


ValueError: Tests directory ./tests does not exist

In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np

## 1. Get the full names of 5 star rating books in the first page.

Check this site [https://books.toscrape.com/catalogue/page-1.html](https://books.toscrape.com/catalogue/page-1.html).

This is just the first page, and there are exactly 50 pages like this, you can just change `page-1` to `page-2` and so on.

For this first question, Just get the names of books from `page-1` which got 5 star rating, as a list.  Names should be full. Find `img` tag for each book by inspecting the corresponding part in page, and check for full name of the book in the `alt` attribute of `img` tag. The variable name should be `fivestar_books`

In [17]:
import requests
from bs4 import BeautifulSoup

# URL of the first page
url = 'https://books.toscrape.com/catalogue/page-1.html'

# Fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all book containers
book_containers = soup.find_all('article', class_='product_pod')

# List to hold the names of 5-star rated books
fivestar_books = []

# Loop through each book container
for book in book_containers:
    # Find the star rating
    star_rating = book.find('p', class_='star-rating')

    # Check if the book has a 5-star rating
    if 'Five' in star_rating['class']:
        # Find the image tag and get the full book name from the alt attribute
        book_name = book.find('img')['alt']
        fivestar_books.append(book_name)

# Output the result
print(fivestar_books)


['Sapiens: A Brief History of Humankind', 'Set Me Free', "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)", 'Rip it Up and Start Again']


In [ ]:
grader.check("q1")

NameError: name 'grader' is not defined

## 2. Combined prices of All Saga books available in the chart

Now Iterate over all 50 pages available to find books from Saga series. There are Saga 1,2,3,5,6 volumes available in the site. These you can get my matching pattern from the titles. Get the total prices of those five books. Return the float value (round nearest to two decimal points) and Don't change the currency. The variable name should be `total_price`

In [59]:
import requests
from bs4 import BeautifulSoup
import re

def get_book_price(book):
    price = book.select_one('div.product_price p.price_color').text
    return float(price[1:])  # Remove the '£' symbol and convert to float

def is_saga_book(title):
    return bool(re.match(r'Saga, Volume \d+', title))

total_price = 0
saga_books_found = set()

base_url = 'https://books.toscrape.com/catalogue/page-{}.html'

for page in range(1, 51):  # Iterate through all 50 pages
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    book_containers = soup.find_all('article', class_='product_pod')

    for book in book_containers:
        book_name = book.find('img')['alt']
        if is_saga_book(book_name):
            saga_volume = re.search(r'Volume (\d+)', book_name).group(1)
            if saga_volume not in saga_books_found and saga_volume in {'1', '2', '3', '5', '6'}:
                saga_books_found.add(saga_volume)
                total_price += get_book_price(book)

    if len(saga_books_found) == 5:  # We've found all the Saga books we're looking for
        break

total_price = round(total_price, 2)
print(f"Total price of Saga books: £{total_price}")

Total price of Saga books: £137.86


In [ ]:
grader.check("q2")

## 3. Convert the above price to Dollars using given API

Convert `total_price` to dollars using the given API. Pass the `total_price` to `pounds` as attribute in `params` for `requests.get`. Return float value, rounding to nearest two decimal points. The variable name should be `price_in_dollars`

In [62]:

def get_book_price(book):
    # Extract the price and handle any issues with whitespace or characters
    price = book.select_one('div.product_price p.price_color').text.strip()
    # Remove any potential non-numeric characters (currency symbols) and convert to float
    return float(re.sub(r'[^\d.]', '', price))

def is_saga_book(title):
    # Match titles that are of the form "Saga, Volume X"
    return bool(re.match(r'Saga, Volume \d+', title))

def convert_to_dollars(pounds):
    api_url = 'https://currency-converter-spark-infra-105732.apps.shift.nerc.mghpcc.org/convert'
    params = {'pounds': pounds}
    response = requests.get(api_url, params=params)
    # Assuming the API returns a JSON with a 'dollars' key containing the converted amount
    if response.status_code == 200:
        data = response.json()
        return round(float(data['dollars']), 2)
    else:
        raise Exception('API request failed')

total_price = 0
saga_books_found = set()

base_url = 'https://books.toscrape.com/catalogue/page-{}.html'

for page in range(1, 51):  # Iterate through all 50 pages
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    book_containers = soup.find_all('article', class_='product_pod')

    for book in book_containers:
        book_name = book.find('img')['alt']
        if is_saga_book(book_name):
            saga_volume = re.search(r'Volume (\d+)', book_name).group(1)
            if saga_volume not in saga_books_found and saga_volume in {'1', '2', '3', '5', '6'}:
                saga_books_found.add(saga_volume)
                total_price += get_book_price(book)

    if len(saga_books_found) == 5:  # We've found all the Saga books we're looking for
        break

# Convert total price from pounds to dollars
price_in_dollars = convert_to_dollars(total_price)
print(f"Total price of Saga books: £{total_price}")
print(f"Total price in Dollars: ${price_in_dollars}")


Total price of Saga books: £137.85999999999999
Total price in Dollars: $175.08


In [ ]:
grader.check("q3")

## NOTE: Q 4, 5, & 6
Check this site https://www.boxofficemojo.com/chart/top_lifetime_gross/ . For questions 4, 5, & 6, focus on the domestic only. Don't change to international.

There are multiple pages, and you can use `params` argument in the `requests.get` to move across the pages, as discusssed in the lab

## 4. List the movies in top 200 that released in or after 2020

For this question, just focus on the first page, which has top 200 (you dont have to pass any headers or params) and get the movie names released in or after 2020, as a list. The variable name should be `movies_list`

In [ ]:
grader.check("q4")

In [21]:
import requests
from bs4 import BeautifulSoup

def get_movies_released_after_2020(url):
    try:
        # Fetch the page content
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.content, 'html.parser')

        # Debug: Print the page content
        print(soup.prettify()[:2000])  # Print the first 2000 characters for inspection

        # Find the table containing the movie data
        table = soup.find('table', {'class': 'a-bordered'})

        if not table:
            print("Table not found")
            return []

        movies_list = []

        rows = table.find_all('tr')

        for row in rows[1:201]:  # Process up to 200 rows (excluding header)
            cols = row.find_all('td')
            if len(cols) > 2:  # Ensure there are enough columns
                movie_name = cols[1].get_text(strip=True)
                release_year = cols[2].get_text(strip=True)  # Assuming release year is in the 3rd column

                # Debug: Print movie name and release year
                print(f"Movie: {movie_name}, Year: {release_year}")

                # Check if the release year is 2020 or later
                if release_year.isdigit() and int(release_year) >= 2020:
                    movies_list.append(movie_name)

        return movies_list

    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return []

# Define the URL for the Box Office Mojo page
url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/'

# Get the list of movies released in or after 2020
movies_list = get_movies_released_after_2020(url)
print("Movies released in or after 2020:")
for movie in movies_list:
    print(movie)


<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo">
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <script type="text/javascript">
   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);


    var ue_err_chan = 'jserr-rw';
(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{};var c=f.logLevel||b

## 5. List the Terminator movies present in the chart


Now Iterate over all 1000 movies available to find Terminator movies in the chart. These you can get my matching pattern from the titles. Get the list of Terminator movies present, as a list. The variable name should be `terminator_movies_list` [ Hint: Just movies which has the word `terminator` present in the title (case insensitive).]

In [25]:
import requests
from bs4 import BeautifulSoup

def get_terminator_movies(url):
    try:
        # Fetch the page content
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.content, 'html.parser')

        # Print the first 2000 characters of the HTML to debug
        print(soup.prettify()[:2000])

        # Find the table containing the movie data
        table = soup.find('table', {'class': 'a-bordered'})

        if not table:
            print("Table not found")
            return []

        terminator_movies_list = []

        rows = table.find_all('tr')

        for row in rows[1:1001]:  # Process up to 1000 rows (excluding header)
            cols = row.find_all('td')
            if len(cols) > 2:  # Ensure there are enough columns
                movie_name = cols[1].get_text(strip=True)

                # Debug: Print each movie name
                print(f"Checking movie: {movie_name}")

                # Check if "Terminator" is in the movie title (case insensitive)
                if 'terminator' in movie_name.lower():
                    terminator_movies_list.append(movie_name)

        return terminator_movies_list

    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return []

# Define the URL for the Box Office Mojo page
url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/'

# Get the list of Terminator movies
terminator_movies_list = get_terminator_movies(url)
print("Terminator movies present in the chart:")
for movie in terminator_movies_list:
    print(movie)



<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo">
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <script type="text/javascript">
   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);


    var ue_err_chan = 'jserr-rw';
(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{};var c=f.logLevel||b

In [ ]:
grader.check("q5")

## Question 6

Calculate the combined gross of all 4 Avengers Movies (Domestic Only)

In [7]:

def get_avengers_gross(url):
    try:
        # Fetch the page content
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the table containing the movie data
        table = soup.find('table', {'class': 'a-bordered'})

        if not table:
            print("Table not found")
            return []

        avengers_movies = ['Avengers', 'Avengers: Age of Ultron', 'Avengers: Infinity War', 'Avengers: Endgame']
        avengers_gross = {}

        rows = table.find_all('tr')

        for row in rows[1:1001]:  # Process up to 1000 rows (excluding header)
            cols = row.find_all('td')
            if len(cols) > 2:  # Ensure there are enough columns
                movie_name = cols[1].get_text(strip=True)
                domestic_gross = cols[3].get_text(strip=True)  # Assuming domestic gross is in the 4th column

                # Extract the numeric part of the gross amount
                domestic_gross_value = ''.join(filter(str.isdigit, domestic_gross))  # Remove commas and currency symbols
                try:
                    domestic_gross_value = int(domestic_gross_value)
                    if any(avenger in movie_name for avenger in avengers_movies):
                        avengers_gross[movie_name] = domestic_gross_value
                except ValueError:
                    continue

        # Calculate the combined gross
        combined_gross = sum(avengers_gross.values())
        return combined_gross

    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return []

# Define the URL for the Box Office Mojo page
url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/'

# Get the combined gross of all Avengers movies
combined_gross = get_avengers_gross(url)
print(f"Combined domestic gross of all Avengers movies: ${combined_gross:,}")



Combined domestic gross of all Avengers movies: $8,064


In [ ]:
grader.check("q6")

## Question 7.

Get the companies belonging to "Oil and gas" industry out of the set of 50 largest companies on this [wikipedia page](https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue):  

In [ ]:
grader.check("q7")

In [69]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue'

# Send a GET request to fetch the page content
response = requests.get(url)

# Parse the page content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table that contains the list of largest companies
table = soup.find('table', {'class': 'wikitable'})

# Read the table into a pandas DataFrame
df = pd.read_html(str(table))[0]

# Flatten the MultiIndex columns
df.columns = [' '.join(col).strip() for col in df.columns.values]

# Limit to the top 50 companies
top_50_companies = df.head(50)

# Filter for companies in the "Oil and gas" industry
oil_and_gas_companies = top_50_companies[top_50_companies['Industry Industry'].str.contains('Oil and gas', case=False, na=False)]

# Show the result
print(oil_and_gas_companies[['Name Name', 'Industry Industry']])


                               Name Name Industry Industry
3                           Saudi Aramco       Oil and gas
4        China Petrochemical Corporation       Oil and gas
5   China National Petroleum Corporation       Oil and gas
12                            ExxonMobil       Oil and gas
13                                 Shell       Oil and gas
24                         TotalEnergies       Oil and gas
26                                    BP       Oil and gas
29                               Chevron       Oil and gas


<ipython-input-69-edc90bee674e>:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
